### 1. Definição do problema
> O objetivo deste trabalho é apresentar uma análise sobre os preços de produtos importados pelo Brasil, comparando os valores declarados numa operação de importação com os preços praticados no comércio internacional.
>Para isso, primeiramente, será realizado um levantamento de preços junto ao mercado mundial para um grupo de mercadorias de alto valor agregado.
> Em seguida, buscar-se-ão os preços desses mesmos produtos declarados em processos de importação de mercadorias.
> Por fim, far-se-á a comparação entre as médias desses valores, visando a uma conclusão quanto a indícios de fraudes - subfaturamento ou superfaturamento - nas operações de importação realizadas pelo importador nacional.

### 2. A coleta de dados
> Os datasets usados neste trabalho foram coletados de fontes diversas. 
> Definiu-se que os sites __www.aliexpress.com__ e __www.alibaba.com__, notórias fornecedores de produtos diversos, serão objeto de webscrapping realizado com a ferramenta de código aberto _Beautifull Soup_, a qual será utilizada para coletar uma gama de aproximadamente 10 mil registros de cada um desses sites.
> Para delimitar o escopo da análise, será definido apenas uma gama de produtos, a qual apresenta uma variação grande de preços: __display lcd para aparelhos celulares__.

__Bold por favor__ <br>
_Italico Olha isso_ <br>
$ a + bX$ <br>
__________ dfjlkaj dlkjf lkajkdj
> item 
>>item2
>>>item3
- item 1
- item 2
1. item 1
2. item 2 <br>
<font color=blue>"font color=blue"</font><br>
paragrafog <br> outro paragrafo<br>
#isto é comentário

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

#bibliotecas para requisição de páginas dinâmicas
from selenium import webdriver 
#from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
import time
import os
    
def alteraIdioma(driver):
    driver.find_element_by_xpath('//*[@id="switcher-info"]').click()#/html/body/div[2]/div/div/div[2]/div[2]/div/div[1]/div[2]/#div[1]/span[2]/span/span[2]/span
    time.sleep(3)
    painel = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div')
    painel.click()
    #painel country selector switcher
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/a[1]/span/span').click()
    time.sleep(3)
    painel.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/div[1]/ul/li[224]").click()
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[4]/button').click()
    driver.refresh()
           
    return driver



#Preparando a aquisição das páginas
    #Criando informação sobre quem está fazendo o scrap
headers = {
    'User-Agent': 'João Lima, from PUC-MG University',
    'From': 'jbetol@gmail.com'
}

#Ainda precisa Verificar com Try Except se driver está dispoível
try: #if os.path.exists('/media/joao/HDLinux/tools/'):
    driver = webdriver.Firefox('/media/joao/HDLinux/tools')
except IOError:
    print("Geckdriver not found!")
finally:
    return    
    


#url_alibaba = "https://www.alibaba.com/trade/search?IndexArea=product_en&CatId=5904002&fsb=y&viewtype=&tab=&SearchScene=&SearchText=phone+display+lcd+screen"

#url_aliexpress = "https://www.aliexpress.com/wholesale?catId=0&initiative_id=SB_20210203140628&SearchText=phone+display+lcd+screen"

url_aliexpress = 'https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=phone+display+lcd+for+screen&ltype=wholesale&SortType=default&page='

url_alibaba = 'https://www.alibaba.com/products/phone_display_lcd_screen/CID5904002.html?spm=a2700.galleryofferlist.0.0.126c1524tZVgH8&IndexArea=product_en&page='

#Vamos pegar 50 páginas de dados e jogar num array
paginas = []
qnt_paginas = 3
passou = False
alibaba = False  # Vamos começar scrapping por Alibaba.com
site_index = 2  # números de sites a ser scrapping
desc_items = {} # Dicionário que vai guardar os dados das páginas in memory
while site_index > 0  :
    site_index = site_index - 1
    for i in range(1,qnt_paginas):
        if not alibaba:
            url = url_aliexpress + str(i)
        else:
            url = url_alibaba + str(i)

        paginas.append(url)

    #loop para navegar em 30 páginas de aquisição de dados
    for pagina in paginas:
        driver.get(pagina)
        #driver.execute_script("arguments[0].setAttribute('lang', 'us')", elemento)#, 'class', 'vote-link up voted')

        if not alibaba and not passou:
            alteraIdioma(driver)
            passou = True

    #Verificar modo silencioso para o geckodriver
        
        req = driver.page_source

        soup = BeautifulSoup(req, "html.parser")

        #Aqui é para páginas estáticas. Não usa o webdriver
        #req = requests.get(url)
        #soup = BeautifulSoup(req.text, "html.parser")
        #Adquirindo a lista dos produtos que se encontra na classe indicada
        if alibaba:
            classes = soup.find('div', class_ = "organic-list app-organic-search__list")
            #Agora vamos buscar os itens da lista
            items =  classes.find_all(class_ = 'organic-gallery-offer-outter J-offer-wrapper')
        else:
            classes = soup.find('ul', class_ = "list-items")
            #Agora vamos buscar os itens da lista
            #items_div =  classes.find_all('div')
            items =  classes.find_all('li', class_ = 'list-item')

        #Para cada item, pegar apenas descrição e o preco e guardar num dicionário
        for item in items:
            if alibaba:
                descricao = item.find('h4').get('title')
                preco = item.find('p', class_ = 'elements-offer-price-normal').get('title')
            else:
                descricao = item.find('a', class_ = 'item-title').get('title')
                preco = item.find('span', class_ = 'price-current').text

            desc_items[descricao] = preco
    alibaba = True
    paginas.clear()

driver.quit()
#transformando o dicionário em DataFrame do Pandas
df = pd.DataFrame(desc_items.items(), columns=['Descrição','Preço']) 
df.info()
# displaying the DataFrame  
print('DataFrame:\n', df) 

# saving the DataFrame as a CSV file 
gfg_csv_data = df.to_csv('lista.csv', header = True) 




FileNotFoundError: [Errno 2] No such file or directory: '/media/joao/HDLinux/tools'

In [105]:
import re
fonte = '$10.63-$16.82 - US $43.77 - 44.27 $15.00-$ 39.81 US $18.49 - US325,30 US $12,70'
fonte_limpa = re.sub(r'[aA-Zz$\-\s+]','',fonte)
#fonte_limpa = re.sub(r'[aA-Zz]','',fonte_limpa)

#print(fonte_limpa +'\n')

print(re.findall(r'(\d+[\.,]\d{1,2})', fonte))


['10.63', '16.82', '43.77', '44.27', '15.00', '39.81', '18.49', '325,30', '12,70']


In [108]:
print(desc_items)

{'Lcd Screen Phone Display Phone Lcd Screen LCD Screen For Phone 7 7plus 8 8plus X Refurbished LCD Display For Phone 7 Plus Screen Replacement OEM Touch Screen LCD': '$14.60', 'Display Phone Lcd Screen Iphone Lcds Mobile Lcd Display Screen Free Shipping Mobile Display Factory 8 Phone Lcd Digitizer Screen For Iphone 8 Mobile Phone Display Lcds Screen Replacement': '$11.53-$12.33', 'Wholesales Mobile Phones LCD Screens TFT mobile lcd Cellphone touch display for iPhone 6SP': '$8.00-$20.60', 'Phone Lcd Screen Display Factory Price Mobile Phone LCD Remplacement Screen Display For Phone 5s': '$5.00-$10.50', 'Gx gx3 Hx Jk Rj Amoled Oled Mobile Phone Lcds Screen Replacement With Touch Screen Digitizer Assembly For Iphone5 6 7 8 10 X xr': '$9.64-$110.98', 'Mobile Accessories Mobile Phone Replacement Touch Screen Display LCD Screen for iPhone': '$1.00', 'Mobile Phone LCD Digitizer Accessories Parts mobile lcd screen Mobile Phone LCDs Touch Display': '$1.00-$5.00', 'lcd screen wholesale for samsu